In [ ]:
!pip install torch transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model ID
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# Load tokenizer and model with necessary configurations
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,   # Using bfloat16 for reduced memory
    device_map="auto"             # Automatically places model on available GPU
)

# Instruction message
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

# Encode input messages
messages = [
    {"role": "user", "content": f"{instruction}"}
]

# If 'apply_chat_template' doesn't exist in the tokenizer, use standard tokenization
try:
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
except AttributeError:
    input_ids = tokenizer(instruction, return_tensors="pt").input_ids.to(model.device)

# Define end-of-sequence tokens
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generate output
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# Decode and display the response
print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))



Exception: data did not match any variant of untagged enum ModelWrapper at line 1251003 column 3